### Map widget to choose region

In [1]:
from worldcereal.utils.map import get_ui_map

m, dc = get_ui_map()
m

Map(center=[51.1872, 5.1154], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [3]:
# retrieve bounding box from drawn rectangle
from worldcereal.utils.map import get_bbox_from_draw

spatial_extent, bbox, poly = get_bbox_from_draw(dc)

2024-10-04 11:49:25.439 | INFO     | worldcereal.utils.map:get_bbox_from_draw:464 - Your area of interest: (4.988426, 51.235841, 5.228854, 51.330326)
2024-10-04 11:49:25.933 | INFO     | worldcereal.utils.map:get_bbox_from_draw:470 - Area of processing extent: 187.53 km²


### Time slider to choose focus time

In [11]:
from utils import suggest_seasons

# This method suggest season start, end and focus time based on default crop calendars
suggest_seasons(spatial_extent)

Start of `tc-annual` season: September 27
End of `tc-annual` season: September 26
Suggested focus time of `tc-annual` season: March 28


In [4]:
# Placeholder interactive slider

focus_time = '2022-06-01'

### Other customization options


In [5]:
postprocess_result = True
map_name = 'my_map'

### Create and run job

In [7]:
from worldcereal.job import generate_map, PostprocessParameters
from worldcereal.seasons import get_processing_dates_for_focus_time

# Get temporal range to generate product
temporal_extent = get_processing_dates_for_focus_time(focus_time)

# Launch the job
job_results = generate_map(
    spatial_extent,
    temporal_extent,
    output_path=f"./{map_name}.tif",
    postprocess_parameters=PostprocessParameters(enable=postprocess_result),
)

2024-10-02 21:17:54.184 | INFO     | worldcereal.seasons:get_processing_dates_for_focus_time:328 - Derived the following period for processing: 2021-12-01 - 2022-11-30
INFO:openeo.rest.connection:Found OIDC providers: ['CDSE']
INFO:openeo.rest.connection:No OIDC provider given, but only one available: 'CDSE'. Using that one.
INFO:openeo.rest.connection:Using default client_id 'sh-b1c3a958-52d4-40fe-a333-153595d1c71e' from OIDC provider 'CDSE' info.
INFO:openeo.rest.connection:Found refresh token: trying refresh token based authentication.
INFO:openeo.rest.auth.oidc:Doing 'refresh_token' token request 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token' with post data fields ['grant_type', 'client_id', 'refresh_token'] (client_id 'sh-b1c3a958-52d4-40fe-a333-153595d1c71e')
INFO:openeo.rest.connection:Obtained tokens: ['access_token', 'id_token', 'refresh_token']
INFO:openeo.rest.auth.config:Storing refresh token for issuer 'https://identity.dataspace.

Authenticated using refresh token.


2024-10-02 21:18:05,410 - openeo_gfmap.utils - INFO - Selected orbit state: DESCENDING. Reason: Orbit has more cumulative intersected area. 1.6111722959235162 > 1.5120232315589912
INFO:openeo_gfmap.utils:Selected orbit state: DESCENDING. Reason: Orbit has more cumulative intersected area. 1.6111722959235162 > 1.5120232315589912


0:00:00 Job 'j-2410027aa7ec40e084303b9388d87e02': send 'start'
0:00:15 Job 'j-2410027aa7ec40e084303b9388d87e02': created (progress 0%)
0:00:20 Job 'j-2410027aa7ec40e084303b9388d87e02': created (progress 0%)
0:00:36 Job 'j-2410027aa7ec40e084303b9388d87e02': created (progress 0%)
0:00:53 Job 'j-2410027aa7ec40e084303b9388d87e02': created (progress 0%)
0:01:04 Job 'j-2410027aa7ec40e084303b9388d87e02': running (progress N/A)
0:01:24 Job 'j-2410027aa7ec40e084303b9388d87e02': running (progress N/A)
0:01:39 Job 'j-2410027aa7ec40e084303b9388d87e02': running (progress N/A)
0:01:58 Job 'j-2410027aa7ec40e084303b9388d87e02': running (progress N/A)
0:02:32 Job 'j-2410027aa7ec40e084303b9388d87e02': running (progress N/A)
0:03:02 Job 'j-2410027aa7ec40e084303b9388d87e02': running (progress N/A)
0:03:40 Job 'j-2410027aa7ec40e084303b9388d87e02': running (progress N/A)
0:04:28 Job 'j-2410027aa7ec40e084303b9388d87e02': running (progress N/A)
0:05:26 Job 'j-2410027aa7ec40e084303b9388d87e02': running (progre

INFO:openeo.rest.job:Downloading Job result asset 'openEO_2020-01-01Z.tif' from https://openeo.creo.vito.be/openeo/jobs/j-2410027aa7ec40e084303b9388d87e02/results/assets/NGZkOWRiOTYtZDYyMC00NDU0LTliZTYtMTRhN2Q4ZTkyMzU3/011c8bd2684181bdf18d497da581e6f5/openEO_2020-01-01Z.tif?expires=1728502385 to my_map.tif
